# Benchmark Repeat OPF

This notebook aims to benchmark the repeative optimal power flow.

In [1]:
import warnings
import logging
import ams
import numpy as np
import pandas as pd

import ams.benchmarks as bp

# Configure AMS logger
ams.config_logger(stream_level=20)

# Display tool versions
_ = bp.get_tool_versions()

ams.config_logger(stream_level=50)

Last run time: 2024-11-06 11:52:33
Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:27:15) [Clang 11.1.0 ]

Tool        Version                      
----------  -----------------------------
ltbams      0.9.10.post153.dev0+gc73b7f8a
cvxpy       1.5.3                        
pandapower  2.14.11                      
PYPOWER     5.1.17                       
gurobipy    11.0.3                       
mosek       10.2.6                       
ecos        2.0.14                       
scs         3.2.7                        
piqp        0.4.2                        
numba       0.60.0                       


In [2]:
%matplotlib inline

In [3]:
cases_lfs = [
    ams.get_case('5bus/pjm5bus_uced.xlsx'),
    ams.get_case('ieee14/ieee14_uced.xlsx'),
    ams.get_case('ieee39/ieee39_uced.xlsx'),
    ams.get_case('npcc/npcc_uced.xlsx'),
    ams.get_case('wecc/wecc_uced.xlsx'),
]

lfs = [ams.load(case, setup=True).EDTSlot.sd.v[:, 0] for case in cases_lfs]

# store loading factor data
lfs_data = pd.DataFrame(data=np.array(lfs).T,
                        columns=['pglib_opf_case5_pjm', 'case14',
                                 'case39', 'npcc', 'wecc'])
lfs_data.to_csv('lfs_data.csv', index=False)

In [4]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logger = logging.getLogger('pandapower').setLevel(logging.ERROR)

cases = [
    './cases/case5.m',
    './cases/case14.m',
    './cases/case39.m',
    './cases/npcc.m',
    './cases/wecc.m',
]

case_names = [case.split('/')[-1].split('.')[0] for case in cases]

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

time_data = np.zeros((n_iters, n_cases, len(bp.cols_time)))
obj_data = np.zeros((n_cases, len(bp.cols_obj)))

In [5]:
for n_case, (case, lf) in enumerate(zip(cases, lfs)):
    print(f'Case: {case}')
    for n_iter in range(n_iters):
        time, obj = bp.test_mtime(case, lf, ignore_dpp=False)
        time_data[n_iter, n_case, :] = time
    obj_data[n_case, :] = obj

Case: ./cases/case5.m
Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25
Case: ./cases/case14.m
Case: ./cases/case39.m
Case: ./cases/npcc.m
Case: ./cases/wecc.m


In [6]:
# NOTE: results structure:
# 1st column: iteration
# 2nd column: cases
# 3rd column: solvers

obj_dcopf = pd.DataFrame(obj_data, columns=bp.cols_obj, index=case_names)

time_dcopf = pd.DataFrame(columns=bp.cols_time, index=case_names)
for case in case_names:
    time_dcopf.loc[case] = time_data[:, case_names.index(case), :].mean(axis=0)

time_dcopf.iloc[:, :] *= 1000        # scale to ms

In [7]:
time_dcopf

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,ams_grb,ams_mosek,ams_piqp,pdp
case5,0.56,1.52,2.18,0.0,0.1,48.52,52.28,36.04,345.87
case14,0.6,1.4,2.18,0.0,0.1,54.87,55.45,39.31,325.07
case39,0.7,1.41,2.33,0.0,0.1,73.16,70.11,45.02,320.7
npcc,1.4,1.4,4.92,0.0,0.1,198.34,188.39,91.15,1301.61
wecc,1.57,1.4,3.46,0.01,0.1,189.02,152.91,95.98,939.71


In [8]:
obj_dcopf.round(2)

,grb,mosek,piqp,pdp
case5,76407795.79,76407795.80,76407795.80,76407795.79
case14,136767.75,136767.75,136767.75,136767.75
case39,643403.27,643403.26,643403.27,643403.27
npcc,16635796.96,16635796.96,16635796.96,16635511.24
wecc,8621568.97,8621568.97,8621568.97,8621568.97


In [ ]:
obj_dcopf.to_csv('../results/results_obj_repeat.csv')
time_dcopf.to_csv('../results/results_time_repeat.csv')